In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import torchtext
import spacy
import string
import gensim
import pickle
import random
from torchtext.data import Field, Dataset, Example
import numpy as np
import math
from torch.autograd import Variable
import time
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
import csv 
import matplotlib.pyplot as plt
from queue import PriorityQueue
#pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
! python -m spacy download xx_ent_wiki_sm
! python -m spacy download en_core_web_sm
! python -m spacy download xx_ent_wiki_sm
import en_core_web_sm
#nlp = en_core_web_sm.load()
import xx_ent_wiki_sm
#nlp = en_core_web_sm.load()

✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_pickle('/content/drive/My Drive/data/en_hi_data/en_hi.pkl')
en = spacy.load('en_core_web_sm')
hin = spacy.load('xx_ent_wiki_sm')
#If you face an error in loading this, run the above cell and restart the session
#Loading spaCy data
#en = en_core_web_sm.load()
#him = xx_ent_wiki_sm.load()

In [4]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"y'all":"you all",
"let's": "let us"
}
#Dictionary mapping contractions to word(s)

In [5]:
#Cleaning data
df['english'] = df['english'].str.lower()
#Convert to lowercase first as all our contractions are in lower case. 
df['english'] = df['english'].replace(contractions,regex=True)
df['english'] = df['english'].str.replace('[.|,|"|,|?|।|\!|©|\-|\_|:|$|~|#|\\|/|@]','')
df['english'] = df['english'].str.replace('[\[|\]|\(|\)]','')
df['english'] = df['english'].str.replace('[0-9]','')
df['english'] = df['english'].str.replace('\n','')
df['english'] = df['english'].str.strip()
#Remove white spaces

df = df[df['english']!='']
df['hindi'] = df['hindi'].str.replace('[.|,|"|,|?|।|\!|©|\-|\_|:|$|~|#|\\|/|@]','')
df['hindi'] = df['hindi'].str.replace('[\[|\]|\(|\)]','')
df['hindi'] = df['hindi'].str.replace('[0-9]','')
df['hindi'] = df['hindi'].str.strip()
df = df[df['hindi']!='']
print(df)

                                 english                           hindi
57382                    who is this man                  ये आदमी कौन है
28149              and the other brother                    और दूसरे भाई
80284           greetings mustakeen bhai            नमस्ते Mustakeen भाई
63596            please reason with them      कृपया उन लोगों के साथ कारण
9459             there is a howling gale                तेज़ हवा आ रही है
...                                  ...                             ...
80140  let us enjoy this life sweetheart   हम इस जीवन जानेमन का आनंद लें
66651             let us get out of here              चलो अब चलो यहाँ से
35946         we are going to the garage      हम गेराज के लिए जा रहे हैं
44917           it is gonna be dark soon  यह जल्द ही अंधेरा होने वाला है
15083           i do not know frau junge        मैं नहीं जानता फ्रॉ जुंग

[9995 rows x 2 columns]


In [6]:
#To tokenize the data
def tokenize_en(text):
  ret = [tok.text for tok in en.tokenizer(str(text))]
  return ret

def tokenize_hi(text):
  ret = [tok.text for tok in hin.tokenizer(str(text))]
  return ret

#Creating fields for use in torchtext.data.Dataset
TEXT_en = torchtext.data.Field(
  tokenize    = tokenize_en, #spaCy tokenizer
  lower       = True,        #lowercase
  init_token  = '<sos>',     #prepend to each example, sos: start of sentence
  eos_token   = '<eos>'      #append to each example, eos: end of sentence.
)

TEXT_hin = torchtext.data.Field(
  tokenize    = tokenize_hi,
  lower       = True,
  init_token  = '<sos>',
  eos_token   = '<eos>'
)

english_data = df['english'].to_list()
print(english_data)
hindi_data = df['hindi'].to_list()

['who is this man', 'and the other brother', 'greetings mustakeen bhai', 'please reason with them', 'there is a howling gale', 'no you do not', 'the violence the rage', 'signal decoy in place', 'you guys come with', 'how long you staying', 'who is this now', 'how is rosie doing jim', 'feet radius clear', 'it is not safe', 'trevor no no do not', 'your passport is ready', 'come on let us go', 'fruit of thy womb', 'agh that is tight', 'is this wool', 'no she is different', 'ask any of them', "so here's the deal", 'how is this possible', 'do not make me wait', 'i am seeing my daughter', 'no doubt about it', 'no it is so quiet', 'forms of exchange offices', 'very very calmly okay', "my husband's still drunk", 'what can i say', 'it is  crackles', 'why are you whispering', 'just reset the controllers', 'that is brothers for you', 'supports it a night', 'well trust me madam', 'shut your mouth', 'it was this morning', 'dauha naer do smrti', 'local lawyer asap', "no you tell o'keefe", 'neural ha

In [7]:
#Splitting the data into testing and training data using sklearn
english_train , english_test, hindi_train, hindi_test = train_test_split(english_data, hindi_data, test_size=0.2, random_state = 27)
train_data,test_data = [],[]
for i in range(len(english_train)):
  temp = [english_train[i], hindi_train[i]]
  train_data.append(temp)
for i in range(len(english_test)):
  temp = [english_train[i], hindi_train[i]]
  test_data.append(temp)

print(len(train_data), len(test_data), len(test_data) + len(train_data) )


7996 1999 9995


In [8]:
def cleaning(data):
  rem = []
  for i in data:
    hin = i[1]
    for j in hin:
      if re.search('^[A-Za-z0-9_.]+$', j):
        rem.append(i)
  for i in rem:
    try:
      data.remove(i)
    except:
     pass

#Extra code to clean the data (if necessary)

print(len(train_data), len(test_data))
cleaning(train_data)
cleaning(test_data)
print(len(train_data), len(test_data))


7996 1999
7546 1883


In [8]:
np.array(train_data).shape

#pair of English and Hindi data to lists of English and Hindi data
#Converting to a list as it's faster to append to a list than 
#to concatenate to a NumPy array
english_data = list(np.array(train_data)[:,0])
extract_data_from = list(np.array(test_data)[:,0])
for data in extract_data_from:
  english_data.append(data)
hindi_data = list(np.array(train_data)[:,1])
extract_data_from = list(np.array(test_data)[:,1])
for data in extract_data_from:
  hindi_data.append(data)

print(len(english_data), len(hindi_data))
#print(english_data)
#print(hindi_data)

9995 9995


In [9]:
for i in range(len(english_data[i])):
  if english_data[i]==None or hindi_data[i]==None:
    print("FAILED CLEANING")

In [10]:
fields = [('src',TEXT_en),('trg',TEXT_hin)]
examples = []
en = spacy.load('en_core_web_sm')
hin = spacy.load('xx_ent_wiki_sm')
for i in range(len(english_data)):
  temp = Example.fromlist([english_data[i],hindi_data[i]],fields=fields)
  print([temp.src,temp.trg])
  examples.append(temp)

Streaming output truncated to the last 5000 lines.
[['get', 'off', 'the', 'street'], ['सड़क', 'से', 'परे']]
[['the', 'pride', 'of', 'berk'], ['बर्क', 'का', 'गौरव']]
[['can', 'you', 'fix', 'my', 'leg'], ['आप', 'मेरा', 'पैर', 'ठीक', 'कर', 'सकते', 'हैं']]
[['what', 'are', 'you', 'saying'], ['हाँ', 'मैं', 'एक', 'भूत', 'देखा', 'है']]
[['i', 'will', 'arrest', 'you', 'now'], ['मैं', 'तुम्हें', 'अब', 'गिरफ्तार', 'करेगी']]
[['i', 'too', 'felt', 'the', 'same'], ['मैं', 'भी', 'यही', 'सोच', 'रहा', 'था']]
[['get', 'out', 'of', 'the', 'way'], ['रास्ते', 'से', 'हट', 'जाओ']]
[['where', 'is', 'she'], ['कहाँ', 'है', 'वो']]
[['her', 'father', 'betrayed', 'me'], ['उसके', 'पिता', 'ने', 'मुझे', 'धोखा', 'दिया']]
[['she', 'is', 'a', 'blank', 'page'], ['वह', 'एक', 'खाली', 'है', 'पेज']]
[['shall', 'we', 'have', 'a', 'smoke'], ['धूम्रपान', 'किया', 'जाए']]
[['should', 'i', 'put', 'it', 'back'], ['वापस', 'रख', 'दूँ']]
[['sir', 'we', 'are', 'at', 'defcon'], ['हम', 'डैफ़कॉन', ' ', 'पर', 'हैं']]
[['remember', 'in', '

In [11]:
train_check,test_check = torchtext.data.Dataset(examples,fields).split(0.8)

In [12]:
#https://torchtext.readthedocs.io/en/latest/data.html#bucketiterator
BATCH_SIZE = 32

train_iterator = torchtext.data.BucketIterator(
    train_check, BATCH_SIZE, sort_within_batch = False, device = device, repeat = False, sort_key = False
)
test_iterator = torchtext.data.BucketIterator(
    test_check, BATCH_SIZE, sort_within_batch = False, device = device, repeat = False, sort_key = False
)

In [13]:
TEXT_en.build_vocab(train_check,test_check)
TEXT_hin.build_vocab(train_check,test_check)
vocab_en = TEXT_en.vocab
vocab_hin = TEXT_hin.vocab
print(len(vocab_en.stoi),len(vocab_hin.stoi))
#print(vocab_en.itos)
#https://torchtext.readthedocs.io/en/latest/vocab.html

4718 5594


In [14]:
def prepare_vectors():
  word2vec_en = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/data/en_hi_data/cc.en.300.vec.gz')
  word2vec_hin = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/data/en_hi_data/cc.hi.300.vec.gz')
  #Vectors are loaded in Google's word2vec C format 
  #Mapping between keys and vectors
  wordVectors_hin = {}
  wordVectors_en = {}

  for i in vocab_en.stoi:
    wordVectors_en[i] = 0
    #Creating an english vector dictionary with the number of keys == length of vocab of the dataset
  for i in wordVectors_en:
    try:
      wordVectors_en[i] = word2vec_en[i]
      print(word2vec_en[i])
    except:
      wordVectors_en[i] = np.random.normal(0, 2*0.08, size=(300))
  print("END")

  for i in vocab_hin.stoi:
    wordVectors_hin[i] = 0
  for i in wordVectors_hin:
    try:
      wordVectors_hin[i] = word2vec_hin[i]
    except:
      wordVectors_hin[i] = np.random.normal(0,2*0.08,size=(300))

  wordVectors_hin_final = []
  wordVectors_en_final = []
  for i in range(len(wordVectors_en)):
    wordVectors_en_final.append(wordVectors_en[vocab_en.itos[i]])
  for i in range(len(wordVectors_hin)):
    wordVectors_hin_final.append(wordVectors_hin[vocab_hin.itos[i]])

  return wordVectors_en_final, wordVectors_hin_final  

def save_vectors(vectors_en,vectors_hin):
  with open('/content/drive/My Drive/data/en_hi_data/Word2Vec_enV2.pkl','wb') as f:
    pickle.dump(vectors_en,f)
  with open('/content/drive/My Drive/Adata/en_hi_data/Word2Vec_hinV2.pkl','wb') as f:
    pickle.dump(vectors_hin,f)

def load_vectors():
  with open('/content/drive/My Drive/data/en_hi_data/Word2Vec_enV2.pkl','rb') as f:
    vectors_en = pickle.load(f)
  with open('/content/drive/My Drive/data/en_hi_data/Word2Vec_hinV2.pkl','rb') as f:
    vectors_hin = pickle.load(f)
  return vectors_en,vectors_hin

In [15]:
def save_results(result):
  with open('/content/drive/My Drive/data/en_hi_data/Results_table.csv','w',newline='') as f:
    Headers = ['source','predicted','actual']
    writer = csv.DictWriter(f,Headers)
    #Writing to a file

    writer.writeheader()
    for j in result:
      #(Un?)tokenizing before writing to the results file
      source = j['src'].replace('<pad>','').replace('<sos>','').replace('<eos>','').strip()
      predicted = j['predicted'].replace('<pad>','').replace('<eos>','').strip()
      actual = j['actual'].replace('<pad>','').replace('<eos>','').strip()
      temp = reversed(source.split())
      source = ' '.join([str(elem) for elem in temp])
      j['src'] = source
      j['predicted'] = predicted
      j['actual'] = actual
    print(result)
    for i in result:
      writer.writerow(i)
    print("-----Written to the file-----")

In [16]:
class encoderRNN(nn.Module):
  def __init__(self, input_size, emb_size, hidden_size, layers, weights):
    super(encoderRNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = layers

    self.embedding = nn.Embedding(input_size,emb_size)
    #self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights))
    #An embedding to represent each word as a vector. Here our embedding length is 300
    self.rnn = nn.LSTM(emb_size, hidden_size, layers)
  
  def forward(self,input):
    #input = input.unsqueeze(0)
    embedded = self.embedding(input)
    #print(np.asarray(embedded).shape)
    #exit(0)
    #embedded = embedded.numpy()
    #embedded = torch.from_numpy(embedded[0, :, :, :])
    #print(embedded[0, :, :, :].shape, "This is ")
    #torch.squeeze(embedded)
    #print(np.asarray(embedded).shape)
    outputs,(hidden,cell) = self.rnn(embedded)
    #Returning the states of the LSTM
    return hidden, cell

class decoderRNN(nn.Module):
  def __init__(self, output_size, emb_size, hidden_size, layers, weights):
    super(decoderRNN,self).__init__()

    self.output_dim = output_size
    self.emb_dim = emb_size
    self.hidden_dim = hidden_size
    self.num_layers = layers

    self.embedding = nn.Embedding(output_size,emb_size)
    #Embedding the hindi words as a vector of length 300.
    #We don't use one-hot as that is very memory inefficient and long vectors are poorly processed by LSTMs
    #self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights))
    self.rnn = nn.LSTM(emb_size, hidden_size, layers)
    #The input to this LSTM is the embedded word vector from the output of the encoder stage
    self.out = nn.Linear(hidden_size, output_size)
  
  def forward(self, input, hidden, cell):
    input = input.unsqueeze(0)
    #print(np.asarray(input).shape)
    embedded = self.embedding(input)
    output,(hidden, cell) = self.rnn(embedded, (hidden, cell))
    prediction = self.out(output.squeeze(0))
    return prediction, hidden, cell

#Note that in the seq2seq model, the length of the output is independent from that of the input.
#i.e. it can map sequences of different lengths to each other.
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super(Seq2Seq,self).__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.device = device

    assert encoder.hidden_size == decoder.hidden_dim, \
      "Hidden dimensions of the encoder and decoder should be the same"
    assert encoder.num_layers == decoder.num_layers, \
      "The encoder and decoder must have an equal number of layers"
  
  def forward(self, src, target, teaching = 0.5):
    batch_size = target.shape[1]
    target_length = target.shape[0]
    target_vocab = self.decoder.output_dim

    outputs = torch.zeros(target_length, batch_size, target_vocab).to(self.device)
    hidden, cell = self.encoder(src)
    input = target[0,:]
    sent = []
    for i in range(batch_size):
      sent.append({'src':'','predicted':'','actual':''})
    for t in range(src.shape[0]):
      for i in range(batch_size):
        curr = sent[i]
        curr['src'] += vocab_en.itos[src[t,i]]+" "
    for t in range(1, target_length):
      output , hidden, cell = self.decoder(input,hidden,cell)
      outputs[t] = output
      #teacher forcing
      force = random.random() < teaching
      top1 = output.argmax(1)
      input = top1 if force else target[t]
      for i in range(batch_size):
        curr = sent[i]
        curr['predicted'] += vocab_hin.itos[top1[i]]+" "
        curr['actual'] += vocab_hin.itos[target[t,i]]+" "
        sent[i] = curr
    return outputs, sent


In [17]:
input_dim = len(vocab_en)
output_dim = len(vocab_hin)
encoder_emb_dim = 300
decoder_emb_dim = 300
hidden_dim = 512
number_layers = 4
wordvectors_en , wordvectors_hin = load_vectors()

encoder = encoderRNN(input_dim, encoder_emb_dim, hidden_dim, number_layers, wordvectors_en)
decoder = decoderRNN(output_dim, decoder_emb_dim, hidden_dim, number_layers, wordvectors_hin)

model = Seq2Seq(encoder,decoder,device).to(device)

In [18]:
def init_weights(m):
  for name,param in m.named_parameters():
    nn.init.uniform_(param.data, -0.08, 0.08)
model.apply(init_weights)
#Initialising weights
#We see the first embedding taking words in the English Dictionary to a word embedding
#The LSTM takes as an input this embedded representation and outputs a hidden state

#The translated ground truth is embedded as a vector and passed to the decoder
#The linear layer maps the representation back to the output dictionary

Seq2Seq(
  (encoder): encoderRNN(
    (embedding): Embedding(4718, 300)
    (rnn): LSTM(300, 512, num_layers=4)
  )
  (decoder): decoderRNN(
    (embedding): Embedding(5594, 300)
    (rnn): LSTM(300, 512, num_layers=4)
    (out): Linear(in_features=512, out_features=5594, bias=True)
  )
)

In [19]:

optimizer = optim.Adam(model.parameters(), lr = 0.001)
TRG_PAD_IDX = vocab_hin.stoi['<sos>']
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def plotTrainTest(train_loss, test_loss):
  epochs = range(1,36)
  plt.plot(epochs, train_loss, 'g', label = 'Training loss')
  plt.plot(epochs, test_loss, 'r', label = 'Testing loss')
  plt.title('Training and Testing Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

def tensorfromSentence(data):
  #Does exactly what the name says
  english = data[0]
  hindi = data[1]
  index_en = [vocab_en.stoi[i] for i in english]
  index_hi = [vocab_hin.stoi[i] for i in hindi]
  en_output = torch.tensor(index_en, dtype = torch.long, device = device).view(-1,1)
  hin_output = torch.tensor(index_hi, dtype = torch.long, device = device).view(-1,1)
  return (en_output,hin_output)

def train(model, iterator, optimizer, criterion, clip = 1):
  model.train()
  total_loss = 0
  for i, batch in enumerate(iterator):
    src = batch.src
    target = batch.trg
    optimizer.zero_grad()
    #Resetting the gradients
    output, sent = model(src, target)
    output_dim = output.shape[-1]
    output = output[1:].view(-1,output_dim).to(device)
    target = target[1:].view(-1).to(device)
    loss = criterion(output, target)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    #Clipping the gradient after the backward pass
    optimizer.step()
    total_loss += loss.item()
  return total_loss/len(iterator)

def evaluate(model,iterator,criterion):
  model.eval()
  epoch_loss = 0
  #torch.no_grad removes the ability to perform backprop (that's fine as we're 'evaluating' our model)
  #but speeds up computation and reduces RAM consumption
  with torch.no_grad():
    for i, batch in enumerate(iterator):
      src = batch.src
      target = batch.trg
      output,sent = model(src,target,0)
      output_dim = output.shape[-1]
      output = output[1:].view(-1, output_dim).to(device)
      target = target[1:].view(-1).to(device)
      loss = criterion(output,target)
      epoch_loss += loss.item()
  return epoch_loss/len(iterator)

In [21]:
np.set_printoptions(precision = 3)
num_epochs = 25
best_valid_loss = float('inf')
train_loss_total = []
test_loss_total = []
for epoch in range(num_epochs):
  start_time = time.time()
  train_loss = train(model , train_iterator, optimizer, criterion)
  valid_loss = evaluate(model, test_iterator, criterion)
  end_time = time.time()
  epoch_mins, epoch_sec = epoch_time(start_time,end_time)
  train_loss_total.append(train_loss)
  test_loss_total.append(valid_loss)
  if valid_loss < best_valid_loss:
    print("YES")
    best_valid_loss = valid_loss
    torch.save(model.state_dict(),'/content/drive/My Drive/data/en_hi_data/Seq2SeqModel_v2.pt')
  print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_sec}s')
  print(f'\tTrain Loss: {train_loss} | Train PPL: {math.exp(train_loss)}')
  print(f'\t Val. Loss: {valid_loss} |  Val. PPL: {math.exp(valid_loss)}')

YES
Epoch: 01 | Time: 5m 40s
	Train Loss: 3.636609450817108 | Train PPL: 37.96290318079343
	 Val. Loss: 3.4195030397839017 |  Val. PPL: 30.554227012186015
YES
Epoch: 02 | Time: 5m 41s
	Train Loss: 3.2430661087036134 | Train PPL: 25.612131058144556
	 Val. Loss: 3.2877298063702054 |  Val. PPL: 26.781994276602337
YES
Epoch: 03 | Time: 5m 37s
	Train Loss: 3.079961133480072 | Train PPL: 21.75755673925072
	 Val. Loss: 3.2171250127610707 |  Val. PPL: 24.95626799175763
YES
Epoch: 04 | Time: 5m 38s
	Train Loss: 2.9360962734222413 | Train PPL: 18.84214796295877
	 Val. Loss: 3.151412733017452 |  Val. PPL: 23.369055508072645
YES
Epoch: 05 | Time: 5m 36s
	Train Loss: 2.798182014942169 | Train PPL: 16.414777807876884
	 Val. Loss: 3.063148960234627 |  Val. PPL: 21.3948226440124
YES
Epoch: 06 | Time: 5m 32s
	Train Loss: 2.7081456961631774 | Train PPL: 15.001432494311487
	 Val. Loss: 3.0309971779111833 |  Val. PPL: 20.71788170626082
YES
Epoch: 07 | Time: 5m 38s
	Train Loss: 2.5400826559066774 | Train P